Importar librerias

In [1]:
from sentence_transformers import SentenceTransformer, losses, InputExample, models, util, quantization
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from torch.utils.data import DataLoader
import torch
from torch import nn

import os

import pandas as pd
import numpy as np

from custom_adapter_module.AdapterModule import AdapterModule

import utils
import importlib
importlib.reload(utils)

/home/estudiante/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'utils' from '/home/estudiante/mlt_project/utils/__init__.py'>

Se configura un modelo de SentenceTransformer personalizado al definir y configurar varios componentes, que incluyen un modelo de incrustación de palabras, un modelo de agrupación, una capa de normalización y un módulo de adaptador. 

El modelo de incrustación de palabras se inicializa con un modelo transformador preentrenado de Sentence Transformers, con ajustes específicos para una longitud máxima de secuencia y sensibilidad a mayúsculas y minúsculas. El modelo de agrupación se configura para usar el promedio de tokens para agrupar, con otros modos de agrupación desactivados. La capa de normalización se define para estandarizar las incrustaciones.

In [2]:
# Carga del modelo de embeddings de palabras
word_embedding_model = models.Transformer(
    model_name_or_path="sentence-transformers/all-MiniLM-L12-v2",  # Modelo base de Sentence Transformers
    max_seq_length=128,  # Longitud máxima de la secuencia
    do_lower_case=False  # No convertir a minúsculas
)

# Definición de los parámetros del modelo de pooling
pooling_model = models.Pooling(
    word_embedding_dimension=384,  # Dimensión de los embeddings de palabras
    pooling_mode_cls_token=False,  # No usar el token CLS para el pooling
    pooling_mode_mean_tokens=True,  # Usar el promedio de los tokens para el pooling
    pooling_mode_max_tokens=False,  # No usar el máximo de los tokens para el pooling
    pooling_mode_mean_sqrt_len_tokens=False,  # No usar el promedio de la raíz cuadrada de la longitud para el pooling
    pooling_mode_weightedmean_tokens=False,  # No usar el promedio ponderado de los tokens para el pooling
    pooling_mode_lasttoken=False,  # No usar el último token para el pooling
    include_prompt=True  # Incluir el prompt en el pooling
)

# Definición del modelo de normalización
normalize = models.Normalize()

# Congelar los pesos del modelo de embeddings de palabras para que no se entrenen
for param in word_embedding_model.parameters():
    param.requires_grad = False

# Configuración del dispositivo para usar GPU si está disponible, de lo contrario usar CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definir el módulo adaptador con las dimensiones de entrada y salida
adapter = AdapterModule(384, 384).to(device)

# Definir el modelo base de Sentence Transformer con las capas de embedding, pooling y normalización
base_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, normalize], device=device)

# Definir el modelo personalizado de Sentence Transformer que incluye el adaptador
custom_domain_model = SentenceTransformer(
    modules=[word_embedding_model, pooling_model, adapter, normalize], device=device
)

custom_domain_model  # Mostrar la arquitectura del modelo personalizado

/home/estudiante/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): AdapterModule(
    (dense1): Linear(in_features=384, out_features=1024, bias=True)
    (dense2): Linear(in_features=1024, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=384, bias=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (3): Normalize()
)

Para evitar que los pesos del modelo de incrustación de palabras se actualicen durante el entrenamiento, todos sus parámetros se congelan. La configuración del dispositivo garantiza que el modelo utilice una GPU si está disponible; de lo contrario, se utilizará la CPU de forma predeterminada. Se crea una instancia de AdapterModule con dimensiones de entrada y salida definidas y se mueve al dispositivo especificado.

Se instancian dos modelos de SentenceTransformer: el modelo base, que incluye la incrustación de palabras, la agrupación y las capas de normalización; y el modelo de dominio personalizado, que adicionalmente incorpora el módulo de adaptador. Esta configuración permite una adaptación flexible de las incrustaciones adaptadas a tareas o dominios específicos. La arquitectura final del modelo de dominio personalizado se muestra para su revisión.

Cargar los conjuntos de datos de entrenamiento y evaluación para tareas de respuesta a preguntas desde los archivos pickled respectivos almacenados en el directorio 'data'.

In [3]:
qa = pd.read_pickle('data/qa_training.pkl')
qa_eval = pd.read_pickle('data/qa_evaluation.pkl')

Crear ejemplos de entrenamiento utilizando los pares de preguntas y respuestas del conjunto de datos qa, donde cada ejemplo consiste en una pregunta (qa[0]) y su respuesta correspondiente (qa[1]).

In [4]:
train_examples = [
    InputExample(texts=[qa[0], qa[1]])
    for qa in qa
]

In [5]:
print("Training lenght: ", len(qa))
print("Validation lenght: ", len(qa_eval['queries']))

Training lenght:  12105
Validation lenght:  1216


Preparar y configurar el proceso de entrenamiento y evaluación para un modelo personalizado de SentenceTransformer. Inicialmente, se crea un conjunto de datos de entrenamiento generando una lista de instancias InputExample, donde cada instancia consiste en un par de textos (pregunta y respuesta). Este conjunto de datos se carga en un "DataLoader", que mezcla los datos en cada época y establece el tamaño del lote en 256.

La pérdida de entrenamiento se define utilizando "MultipleNegativesSymmetricRankingLoss", que es adecuada para tareas de recuperación de información que involucran pares de texto positivos. Se configura un evaluador utilizando "InformationRetrievalEvaluator", que evalúa el rendimiento del modelo en un conjunto de consultas y corpus, con la función de puntuación principal especificada como "dot_score".

In [6]:
# Crear el dataset de entrenamiento
# Se crea una lista de InputExample, donde cada ejemplo es un par de textos (pregunta y respuesta)
train_dataset = [
    InputExample(texts=[qa[0], qa[1]])
    for qa in qa
]

# Crear el DataLoader para el dataset de entrenamiento
# shuffle=True permite mezclar los datos en cada época
loader = DataLoader(train_dataset, shuffle=True, batch_size=256)

# Definir la función de pérdida
# MultipleNegativesSymmetricRankingLoss es adecuada para tareas de recuperación de información con pares positivos
train_loss = losses.MultipleNegativesSymmetricRankingLoss(custom_domain_model)

# Definir el evaluador
# InformationRetrievalEvaluator evalúa el modelo en un conjunto de consultas y corpus
evaluator = InformationRetrievalEvaluator(
    qa_eval['queries'], 
    qa_eval['corpus'], 
    qa_eval['relevant_docs'], 
    name='qa_eval', 
    main_score_function='dot_score'
)

# Definir el número de épocas y los pasos de calentamiento
epochs = 500
warmup_steps = int(len(loader) * epochs * 0.1)

El número de épocas de entrenamiento se establece en 500 y los pasos de calentamiento se calculan como el 10% del total de pasos de entrenamiento, determinados por la longitud del DataLoader y el número de épocas. Esta configuración asegura que el modelo se prepare y evalúe adecuadamente durante el entrenamiento.

In [7]:
# Entrenar el modelo
# fit() entrena el modelo con los objetivos de entrenamiento y evaluador
custom_domain_model.fit(
    train_objectives=[(loader, train_loss)],  # Objetivos de entrenamiento: DataLoader y función de pérdida
    epochs=epochs,  # Número de épocas
    warmup_steps=warmup_steps,  # Número de pasos de calentamiento
    output_path='results/domain_adaptation_model',  # Ruta de salida para guardar el modelo entrenado
    show_progress_bar=True,  # Mostrar barra de progreso durante el entrenamiento
    save_best_model=True,  # Guardar el mejor modelo según la evaluación
    use_amp=True,  # Habilitar Mixed Precision 
    evaluator=evaluator,  # Evaluador para evaluar el modelo durante el entrenamiento
    evaluation_steps=50,  # Evaluar el modelo cada 50 pasos
)


Epoch: 100%|██████████████████████████████████| 500/500 [32:14<00:00,  3.87s/it]


## Evaluación del modelo base y del modelo personalizado

In [8]:
custom_domain_model = SentenceTransformer('./results/domain_adaptation_model')

Evaluar la tasa de aciertos en k=10 para tanto el modelo base como el modelo de dominio personalizado utilizando los datos de evaluación de preguntas y respuestas, e imprimir los resultados para comparación.

In [9]:
hit_rate = utils.hit_rate_at_k(qa_eval['queries'], qa_eval['corpus'], qa_eval['relevant_docs'], k=10, model=base_model)
print(f"Hit Rate @ 10 (base model): {round(hit_rate, 2)}")


hit_rate = utils.hit_rate_at_k(qa_eval['queries'], qa_eval['corpus'], qa_eval['relevant_docs'], k=10, model=custom_domain_model)
print(f"Hit Rate @ 10 (custom model): {round(hit_rate, 2)}")

Hit Rate @ 10 (base model): 0.1
Hit Rate @ 10 (custom model): 0.96


Evaluar la precisión media promedio (MAP) en k=100 tanto para el modelo base como para el modelo de dominio personalizado utilizando el evaluador, e imprimir los resultados para comparación.

In [10]:
eva_base_model = evaluator(base_model, output_path='results/base_model/')
print("MAP @ 100 (base model): ", round(eva_base_model, 3))

eva_custom_model = evaluator(custom_domain_model, output_path='results/custom_model/')
print("MAP @ 100 (custom model): ", round(eva_custom_model, 3))

MAP @ 100 (base model):  0.002
MAP @ 100 (custom model):  0.038


Cargar los resultados de evaluación desde archivos CSV tanto para el modelo base como para el modelo de dominio personalizado, agregar una columna para indicar el tipo de modelo y concatenar los resultados en un solo DataFrame para su comparación.

In [11]:
base_model_eval = pd.read_csv('results/base_model/Information-Retrieval_evaluation_qa_eval_results.csv')
base_model_eval['tipo'] = 'base_model'
custom_model_eval = pd.read_csv('results/custom_model/Information-Retrieval_evaluation_qa_eval_results.csv')
custom_model_eval['tipo'] = 'custom_model'

pd.concat([base_model_eval, custom_model_eval]).to_csv('results/eval_comparation.csv', index=False)

pd.concat([base_model_eval, custom_model_eval])


,epoch,steps,cos_sim-Accuracy@1,cos_sim-Accuracy@3,cos_sim-Accuracy@5,cos_sim-Accuracy@10,cos_sim-Precision@1,cos_sim-Recall@1,cos_sim-Precision@3,cos_sim-Recall@3,...,dot_score-Precision@3,dot_score-Recall@3,dot_score-Precision@5,dot_score-Recall@5,dot_score-Precision@10,dot_score-Recall@10,dot_score-MRR@10,dot_score-NDCG@10,dot_score-MAP@100,tipo
0,-1,-1,0.035362,0.055921,0.069901,0.100329,0.035362,0.001052,0.018640,0.001566,...,0.018640,0.001566,0.013980,0.001918,0.010033,0.002831,0.051700,0.013919,0.001890,base_model
1,-1,-1,0.035362,0.055921,0.069901,0.100329,0.035362,0.001052,0.018640,0.001566,...,0.018640,0.001566,0.013980,0.001918,0.010033,0.002831,0.051700,0.013919,0.001890,base_model
0,-1,-1,0.514803,0.754934,0.860197,0.943257,0.514803,0.028315,0.251645,0.042509,...,0.251645,0.042509,0.172039,0.047570,0.094326,0.052310,0.657860,0.167424,0.036689,custom_model
1,-1,-1,0.541118,0.773849,0.875000,0.956414,0.541118,0.030398,0.257950,0.044261,...,0.257950,0.044261,0.175000,0.048883,0.095641,0.053223,0.678546,0.171781,0.038399,custom_model


### Comparando la representación de autores

In [12]:
# Asumiendo que los embeddings están normalizados
question1 = "Which authors are working in Artificial Intelligence?"
question2 =  "Which authors are working in Biology?"
ai_author = "Kudenko D."
bio_author = "Susan H. Fisher"

emb_q1 = custom_domain_model.encode(question1)  # el embedding está normalizado
emb_q2 = custom_domain_model.encode(question2)  # el embedding está normalizado
autai_emb = custom_domain_model.encode(ai_author)
autbio_emb = custom_domain_model.encode(bio_author)


print("Which authors are working in Artificial Intelligence?", autbio_emb @ emb_q1,"(Susan) --", autai_emb @ emb_q1, "(Kudenko)")
print("Which authors are working in Biology?", autbio_emb @ emb_q2, "(Susan) --", autai_emb @ emb_q2, "(Kudenko)")

Which authors are working in Artificial Intelligence? -0.012009517 (Susan) -- 0.16673326 (Kudenko)
Which authors are working in Biology? 0.19324037 (Susan) -- 0.054803345 (Kudenko)


### El modelo personalizado mantiene las capacidades originales

Codifica entradas de texto de ejemplo, incluyendo el título de un artículo, nombres de autores y varios conceptos, utilizando tanto el modelo de dominio personalizado como el modelo base. Además, se calcula el producto punto entre los vectores codificados para medir la similitud entre diferentes pares de conceptos y entre el artículo y un concepto. Imprime los puntajes de similitud para cada comparación para ver las diferencias.

In [13]:
paper = "Composable Lightweight Processors"

concept1 = "shark"
concept2 = "ocean"
concept3 = "strawberry"

In [14]:
custom_paper = custom_domain_model.encode(paper)

custom_concept1 = custom_domain_model.encode(concept1)
custom_concept2 = custom_domain_model.encode(concept2)
custom_concept3 = custom_domain_model.encode(concept3)

# Imprimir los resultados y explicaciones
print(f"Producto punto entre dos conceptos (shark y ocean): {np.dot(custom_concept1, custom_concept2)}")
print(f"Producto punto entre dos conceptos (shark y strawberry): {np.dot(custom_concept1, custom_concept3)}")
print(f"Producto punto entre el documento y un concepto (ocean): {np.dot(custom_paper, custom_concept2)}")

Producto punto entre dos conceptos (shark y ocean): 0.6055551767349243
Producto punto entre dos conceptos (shark y strawberry): 0.2159203290939331
Producto punto entre el documento y un concepto (ocean): -0.1352805197238922


In [15]:
base_paper = base_model.encode(paper)

base_concept1 = base_model.encode(concept1)
base_concept2 = base_model.encode(concept2)
base_concept3 = base_model.encode(concept3)  

# Imprimir los resultados y explicaciones
print(f"Producto punto entre dos conceptos (shark y ocean): {np.dot(base_concept1, base_concept2)}")
print(f"Producto punto entre dos conceptos (shark y strawberry): {np.dot(base_concept1, base_concept3)}")
print(f"Producto punto entre el documento y un concepto (ocean): {np.dot(base_paper, base_concept2)}")

Producto punto entre dos conceptos (shark y ocean): 0.5232284069061279
Producto punto entre dos conceptos (shark y strawberry): 0.23324596881866455
Producto punto entre el documento y un concepto (ocean): -0.08226228505373001
